In [158]:
import random

def is_prime(n, k=10):
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def find_p_q(print_result=True):
    lower_bound = 2**255
    upper_bound = 2**256 - 1
    primes = []
    while len(primes) < 2:
        candidate = random.randint(lower_bound, upper_bound)
        small_primes = [2, 3, 5, 7]
        if any(candidate % p == 0 for p in small_primes):
            if print_result:
                print(f"{candidate} - test failed")
                continue

        if is_prime(candidate):
            if print_result:
                print(f"{candidate} - test passed")
            primes.append(candidate)
        else:
            if print_result:
                print(f"{candidate} - test failed")
    num1, num2 = primes
    if print_result:
        print("\nNumbers, that passed the test:")
        print(f"num1 = {num1}")
        print(f"num2 = {num2}\n\n\n")
    return num1, num2



p, q = find_p_q()
p1, q1 = find_p_q()

pq = p * q
p1q1 = p1 * q1
if pq > p1q1:
    p, q, p1, q1 = p1, q1, p, q

print("\nFinal primes:")
print(f"p = {p}")
print(f"q = {q}")
print(f"p1 = {p1}")
print(f"q1 = {q1}")


107212521660046470580320042797987103276397477435805190142736928141534659048017 - test failed
92368858577131830278821442224933098194435468707804273734008850416993270027459 - test failed
86297952729119221704149927064625929817075841716990097646980664509631860703905 - test failed
67134748848695835897033492232112652468204368295142459044721818594186410170189 - test failed
94450202655066910995503982251091465319538647168807959513704043510225808491473 - test failed
63296076705446751443223536820846226031693083511459015709166172378492906116655 - test failed
65315278447676279416396110638801770148977670577315062941527057138585156310995 - test failed
59570887146794443376804353802450079329104418627810052099816879279068648541318 - test failed
105651623072883042268064998678058040129248597147096693595566997042104123368229 - test failed
62512935634298350637830790976223813638632159040914832802775205511471796946961 - test failed
112980003080306455924192602539229942031056732803200820006891209250929033440629

In [159]:
from math import gcd
from sympy import mod_inverse

def generate_keys(p, q):
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = random.randint(2, phi_n - 1)
    while gcd(e, phi_n) != 1:
        e = random.randint(2, phi_n - 1)
    d = mod_inverse(e, phi_n)
    return (n, e), d

public_key, private_key = generate_keys(p, q)
public_key1, private_key1 = generate_keys(p1, q1)

print(f"Abonent A (p, q): {p}, {q}")
print(f"public_key (n, e): {public_key}")
print(f"private_key (d): {private_key}")

print(f"\nAbonent B (p1, q1): {p1}, {q1}")
print(f"public_key (n1, e1): {public_key1}")
print(f"private_key (d1): {private_key1}")

Abonent A (p, q): 67845620259981189366585723195734144770483020503469685846483122172578848654491, 114385468156576160794849508131159213887850041436772956075857947681549891342467
public_key (n, e): (7760553035811236763946645705614789278280652328595974712109035790412275639319732725395241279998067013415608548284739965428235452797732480427398272638569297, 288976268753249784722168377786300411425816860014700936549357176820728993513147548401024822637191612978317565358257499021103424642680778601983413649225497)
private_key (d): 1081174012268271874278461572483384736188491632039049931375893345172577153307857141077093916564451286205744507542764930426365333471345641940971297802559313

Abonent B (p1, q1): 91063306518095577083619788945055598688778957653014223597442823931269532793079, 107146066373338155100823897361216599784275260701096629187947543255092172013131
public_key (n1, e1): (975707508436350574935624775702536532262803400161900613146300562716298972023646352009474675127699926365159022932293303534

In [167]:
def rsa_encrypt(message, n, e):
    message_encrypted = pow(message, e, n)
    return message_encrypted

def rsa_decrypt(message_encrypted, n_self, d_self):
    message_decrypted = pow(message_encrypted, d_self, n_self)
    return message_decrypted

def rsa_sign(message, n, e, n_self, d_self):
    auth = rsa_encrypt(message, n_self, d_self)
    auth_encrypted = rsa_encrypt(auth, n, e)
    return auth_encrypted

def rsa_verify(auth_encrypted, n, e, n_self, d_self):
    auth_decrypted = rsa_decrypt(auth_encrypted, n_self, d_self)
    auth = rsa_decrypt(auth_decrypted, n, e)
    return auth

In [161]:
def send_key(n, e, n_self, d_self):
    message, _ = find_p_q(print_result=False)
    print(f"Message to send: {message}")
    k = rsa_encrypt(message, n, e)
    print(f"Encrypted Message: {k}")
    s = rsa_sign(message, n, e, n_self, d_self)
    print(f"Encrypted Authentificator: {s}\n")
    return k, s

def receive_key(k, s, n, e, n_self, d_self):
    message = rsa_decrypt(k, n_self, d_self)
    auth = rsa_verify(s, n, e, n_self, d_self)
    if message != auth:
        print('Verification failed')
    else:
        print('Verification complete', f'\nReceived message: {message}')



public_key, private_key = generate_keys(p, q)
public_key1, private_key1 = generate_keys(p1, q1)
print(f"Abonent A (n, e), d: {public_key}, {private_key}")
print(f"Abonent B (n1, e1), d1: {public_key1}, {private_key1}\n")

print('On abonent A side')
k, s = send_key(public_key1[0], public_key1[1], public_key[0], private_key)

print('On abonent B side')
receive_key(k, s, public_key[0], public_key[1], public_key1[0], private_key1)

Abonent A (n, e), d: (7760553035811236763946645705614789278280652328595974712109035790412275639319732725395241279998067013415608548284739965428235452797732480427398272638569297, 4033630324494690032409059260043258648857780696828748515745362311679547514831349548452467648474473312105405951417675409489224388697158000025887567154614319), 7415871350980227994036271963820678315811140076306508014154882865959872497791673564755154248819917687334389957740884332840700963672475244402005617693674019
Abonent B (n1, e1), d1: (9757075084363505749356247757025365322628034001619006131463005627162989720236463520094746751276999263651590229322933035343424362407271013550248912693920349, 1428635004927733390957986995747791421258667668635465627418376607798754637303262216091693756299628872894506930957848644228034460104553033389259653179026767), 4226411973263739067639006427557729530350184381154731648352734752317436535702344227535607565990562509875174948796904171702781307212379179212698899718278503

On abonent A sid

In [170]:
text = rsa_decrypt(6555481942511649078052072557715551344966157822262246669835605516120112741374532452035868901328004213657174405512356815257222390652492135784962377580740436,
                   9757075084363505749356247757025365322628034001619006131463005627162989720236463520094746751276999263651590229322933035343424362407271013550248912693920349,
                   4226411973263739067639006427557729530350184381154731648352734752317436535702344227535607565990562509875174948796904171702781307212379179212698899718278503)
text = hex(text)[2:]
print(text)

911


In [171]:
k = 2143489918377023845215663663890954153157698992738023612244226517251248799772527347989553940090145741875325885005773400890848511535207224147361908073534784
s = 1977759463999683815271502386775395724777250403766209586087726187144634137974622067779947484236207621872477558227903064171112585272001748516186337818928219
n = 77244726599033665206644363132815992345328773232384729189197344850445857360893
e = 65537
n_self = 9757075084363505749356247757025365322628034001619006131463005627162989720236463520094746751276999263651590229322933035343424362407271013550248912693920349
d_self = 4226411973263739067639006427557729530350184381154731648352734752317436535702344227535607565990562509875174948796904171702781307212379179212698899718278503


text = receive_key(k, s, n, e, n_self, d_self)

Verification complete 
Received message: 15874756307178248791
